In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
# 시각화 결과가 선명하게 표시되도록
%config InlineBackend.figure_format = 'retina'

df = pd.read_csv("word2vec_wrangling.csv")

In [2]:
df.head()

,exercise_name,Content_txt
0,PT,"💯 What I try to educate my clients around, doe..."
1,검도,#20200115\n저녁 초대!\n와인잔 속에 비치는\n모든 것들이 화려한\n도심속...
2,기구필라테스,#오늘의동작\n캐딜락 동작의 완성 '행잉'\n⠀\n중력을 이용해 척추를 늘려주고\n...
3,다빈치바디보드,#mbn생생정보마당 \n#고투\n#고투GX\n#다빈치바디보드\n#생방송 #GOTOL...
4,드럼스틱,#드럼스틱 #고무팁 #테크라스틱 #전자드럼용스틱\n\n1. 전자드럼타격시 덜 시끄럽...


In [3]:
import soynlp
import gensim

In [4]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [5]:
def open_file(exercise_name):
    file_name = "#" + exercise_name + "_sum.txt"
    file_1 = "/Users/noopy/FitCuration/" + file_name
    text = open(file_1, 'r', -1, "UTF-8", errors="ignore").read()
    return text

In [6]:
sample_content = open_file("자이로토닉")

In [7]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'   자이로토닉  gyrotonic 비교할 수 없는 레슨 퀄리티     월 프롬과 함께하세요   위례신도시 No      월 이벤트  신규 개인레슨   회   만원 선착순   분   신규 그룹레슨 최대    할인 개월수 별로 할인율 상이    인 이상 단체등록 시 신규클래스 오픈      인 소수 그룹레슨 미국 BBU 장비        개인레슨 미국 Gratz  BBU  Gyrotonic 장비     클래식필라테스  자이로토닉    프롬필라테스 위례동로     우성메디피아  층                  안녕하세요  에코필라테스 플러스입니다     설 연휴 전까지 열심히 운동하러 오시는 우리 멋찐 회원님들을 맞이하려 저는 출근하자마자 부랴부랴 청소를 끝마쳤답니다      어서 센터를 뜨겁게 달구어 주세요     오늘도 평화로운 에코필라테스 플러스와 함께        자세한 문의 및 상담                 카카오 오픈채팅   에코필라테스플러스   blog naver com ecopilatesplus   첨먹어보는 춰컬릿    사만다 에듀케이터  samantha jrobinsonhk 의 달달한간식   맛있다   ㅎㅎ자꾸먹게되는군요      아치앤컬  필라테스  폴스타필라테스  자이로토닉  자이로토닉   자이로키네시스  자이로키네시스   청담필라테스  청담자이로토닉  부산필라테스  부산자이로토닉  archncurl  pilates  polestarpilates  gyrotonic  gyrokinesis 효지원장님  misa limepilates 의 선물    교육듣느라 안그래도 정신없을텐데 요렇게 또 챙겨주니 넘 고마워요 이쁘게 잘쓸게용         아치앤컬  필라테스  폴스타필라테스  자이로토닉  자이로토닉강사  자이로키네시스  자이로키네시스   청담필라테스  부산필라테스  청담자이로토닉  부산자이로토닉  archncurl  pilates  polestarpilates  gyrotonic  gyrokinesis  지유필라테스 자이로토닉 G YOU

In [8]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [9]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = df['Content_txt'].apply(preprocessing)

CPU times: user 948 ms, sys: 13.1 ms, total: 961 ms
Wall time: 1.15 s


In [10]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

/Users/noopy/opt/anaconda3/lib/python3.7/site-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [11]:
# 전처리 이후의 샘플 텍스트로 토큰화
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['자이로토닉', 'gyrotonic', '비교할', '수', '없는', '레슨', '퀄리티', '월', '프롬과', '함께하세요']

In [12]:
print(len(tokened_content))

43663


In [22]:
# 초기화 및 모델 학습
from gensim.models import word2vec

%time tokens = sentences.apply(tokenizer.tokenize)

# 모델 학습
model = word2vec.Word2Vec(tokens, min_count=1)

model

2020-02-03 00:32:23,593 : INFO : collecting all words and their counts
2020-02-03 00:32:23,594 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


CPU times: user 16.6 s, sys: 23.9 ms, total: 16.6 s
Wall time: 16.6 s


2020-02-03 00:32:24,153 : INFO : collected 324341 word types from a corpus of 2300597 raw words and 61 sentences
2020-02-03 00:32:24,154 : INFO : Loading a fresh vocabulary
2020-02-03 00:32:25,685 : INFO : effective_min_count=1 retains 324341 unique words (100% of original 324341, drops 0)
2020-02-03 00:32:25,686 : INFO : effective_min_count=1 leaves 2300597 word corpus (100% of original 2300597, drops 0)
2020-02-03 00:32:26,489 : INFO : deleting the raw counts dictionary of 324341 items
2020-02-03 00:32:26,494 : INFO : sample=0.001 downsamples 9 most-common words
2020-02-03 00:32:26,495 : INFO : downsampling leaves estimated 2287695 word corpus (99.4% of prior 2300597)
2020-02-03 00:32:27,172 : INFO : estimated required memory for 324341 words and 100 dimensions: 421643300 bytes
2020-02-03 00:32:27,173 : INFO : resetting layer weights
2020-02-03 00:33:14,694 : INFO : training model with 3 workers on 324341 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2

In [23]:
stopwords_Twitter = ["입니다","있는","있습니다","같은","안녕하세요","고마워요","있어요","있게"\
                     ,"있도록","부탁드립니다","하는","합니다","할","하세요","하기","해","됩니다","하여",'잘','된','되고','되어','되었습니다',"없는","드립니다"\
                    ,"되기","하시는","하고","않을","같다","싶다","이런","저런","그런",'바랍니다'\
                    ,"했습니다","했다","해드립니다","하신","하실","않고","해요","가능합니다","하고싶으신"\
                    ,"않으며","주세요","오세요"]

stopwords_mecab = ['수','퀄리티','도시','분','전문','스타','년','원',\
                   '월','화','수','목','금','시','앤','일','그램','문'] 

stopwords = stopwords_mecab + stopwords_Twitter

In [24]:
# 모델 이름을 지정하고 저장한다.
model_name = 'my_second_model'
model.save(model_name)

2020-02-03 00:33:25,025 : INFO : saving Word2Vec object under my_second_model, separately None
2020-02-03 00:33:25,026 : INFO : storing np array 'vectors' to my_second_model.wv.vectors.npy
2020-02-03 00:33:25,323 : INFO : not storing attribute vectors_norm
2020-02-03 00:33:25,324 : INFO : storing np array 'syn1neg' to my_second_model.trainables.syn1neg.npy
2020-02-03 00:33:25,607 : INFO : not storing attribute cum_table
2020-02-03 00:33:26,140 : INFO : saved my_second_model


In [8]:
# 저장한 모델은 여기서 꺼내면 된다
import pandas
from gensim.models import Word2Vec

model = Word2Vec.load("my_second_model")

In [9]:
# 단어 사전 수
len(model.wv.vocab)

324341

In [26]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:15]

['운동',
 '다이어트',
 '요가',
 '필라테스',
 '월',
 '시',
 '일',
 '운동하는여자',
 '수',
 '일상',
 'a',
 '더',
 't',
 '분',
 '함께']

In [25]:
# Counter로 자주 등장하는 단어 보기
from collections import Counter
dict(Counter(vocab).most_common(10))

{'운동': <gensim.models.keyedvectors.Vocab at 0x7f7ef474be50>,
 '다이어트': <gensim.models.keyedvectors.Vocab at 0x7f7ef474bf10>,
 '요가': <gensim.models.keyedvectors.Vocab at 0x7f7ef48cf710>,
 '필라테스': <gensim.models.keyedvectors.Vocab at 0x7f7e71733e10>,
 '월': <gensim.models.keyedvectors.Vocab at 0x7f7ef48fc550>,
 '시': <gensim.models.keyedvectors.Vocab at 0x7f7ef49d8b10>,
 '일': <gensim.models.keyedvectors.Vocab at 0x7f7ef49eb210>,
 '운동하는여자': <gensim.models.keyedvectors.Vocab at 0x7f7ef474bed0>,
 '수': <gensim.models.keyedvectors.Vocab at 0x7f7e71782150>,
 '일상': <gensim.models.keyedvectors.Vocab at 0x7f7e71733f10>}

In [12]:
# 가장 적게 등장하는 단어
min(vocab, key=vocab.get)

'encourage'

In [24]:
model.wv.most_similar('힙레')

[('씨티포레스티벌', 0.9979838728904724),
 ('기능성', 0.9974091649055481),
 ('판교', 0.9973074197769165),
 ('W', 0.9963376522064209),
 ('가희와', 0.9961326718330383),
 ('근막', 0.995990514755249),
 ('김포요가', 0.9957764148712158),
 ('점핑핏', 0.9956825971603394),
 ('외에도', 0.9956777095794678),
 ('힙합발레', 0.995653510093689)]

In [14]:
model.wv.most_similar('복근')

[('근육', 0.9995039701461792),
 ('가즈아', 0.9991205930709839),
 ('유아체육', 0.9989883899688721),
 ('폴댄스', 0.9987660646438599),
 ('광주', 0.9987014532089233),
 ('복근운동', 0.9985569715499878),
 ('태권도', 0.9985353350639343),
 ('모델', 0.9984759092330933),
 ('스파링', 0.9983445405960083),
 ('MMA', 0.9982408881187439)]

In [15]:
model.wv.most_similar('척추')

[('엄청', 0.998993992805481),
 ('등등', 0.9989675283432007),
 ('다리', 0.9986965656280518),
 ('하네요', 0.9986326694488525),
 ('힘들게', 0.9986114501953125),
 ('걱정', 0.9986083507537842),
 ('다행히', 0.9985339641571045),
 ('고마워', 0.9985274076461792),
 ('힘들고', 0.9983700513839722),
 ('평생', 0.9983628988265991)]

In [16]:
model.wv.most_similar(positive=['척추','건강'], negative=['우한'])

[('타바타운동', 0.9963933229446411),
 ('퍼스널트레이너', 0.9963120222091675),
 ('근력운동', 0.9958586692810059),
 ('웨이트', 0.9956011772155762),
 ('안다르', 0.9955255389213562),
 ('홈트', 0.995509922504425),
 ('바른자세', 0.9954653382301331),
 ('자기관리', 0.9952776432037354),
 ('근육기능운동', 0.9952510595321655),
 ('체력증진', 0.9951571822166443)]

In [17]:
model.wv.most_similar(positive=['척추','건강'])

[('운동영상', 0.9981263875961304),
 ('대한폴댄스연맹', 0.9980324506759644),
 ('팍팍', 0.9976912140846252),
 ('축구', 0.9974087476730347),
 ('아침운동', 0.9973322153091431),
 ('체력', 0.9972175359725952),
 ('운동시작', 0.9970873594284058),
 ('pointgym', 0.9970570802688599),
 ('점핑', 0.9969454407691956),
 ('작심삼일', 0.9968618750572205)]

In [18]:
model.wv.similarity('척추', '검도')

0.9884146

In [21]:
exercise_to_loop = df["exercise_name"].to_list()


exercise_to_loop.sort()
exercise_to_loop[:10]

['PT',
 '검도',
 '기구필라테스',
 '다빈치바디보드',
 '드럼스틱',
 '등산',
 '라틴댄스',
 '매트필라테스',
 '뮤직복싱',
 '바차타']

In [22]:
vector_val = []

def tuple(exercise_name,word):
    return (word, model.wv.similarity(exercise_name,word))

def show_similarity_for_each_exercise(word):
    for item in exercise_to_loop:
        vector_val.append(tuple(word, item))
    return vector_val

vector_list = show_similarity_for_each_exercise('복근')

In [23]:
def Sort(tup): 
    return(sorted(tup, key = lambda x: float(x[1]), reverse = True)) 

Sort(vector_list)[:10]

[('폴댄스', 0.9987661),
 ('플라잉필라테스', 0.9956836),
 ('크로스핏', 0.99524415),
 ('발레', 0.99482024),
 ('사이클', 0.9946428),
 ('요가쿠아', 0.9944515),
 ('조깅', 0.9943916),
 ('복싱', 0.99346507),
 ('수영', 0.99276334),
 ('패들보드', 0.9924408)]